## Implementing granular access control to Offline Feature Store and Feature Groups using AWS Lake Formation

Using Row- and Cell-level access using AWS Lake Formation. Extract the data from Feature Store using Athena query

![Granular Access using Lake Formation](../images/m9_nb1_lakeformation_architecture.png "Granular Access using Lake Formation")

In [ ]:
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker import get_execution_role
import sagemaker
import logging
import boto3
import pandas as pd
import time
import re
import os
import sys
import pandas as pd
sys.path.append('..')
from utilities import Utils
from utilities import feature_store_helper

In [ ]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Initialize default parameters

In [ ]:
sagemaker_session = sagemaker.Session()
account_id = sagemaker_session.account_id()
role = sagemaker.get_execution_role()
region = sagemaker_session.boto_region_name
default_bucket = sagemaker_session.default_bucket()
s3_client = boto3.client('s3', region_name=region)
query_results= 'sagemaker-featurestore-workshop'
prefix = 'sagemaker-feature-store'

#### Initialize boto3 runtime

In [ ]:
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = sagemaker.Session(boto_session=boto_session, 
                                          sagemaker_client=sagemaker_client, 
                                          sagemaker_featurestore_runtime_client=featurestore_runtime)

#### Retrieve the orders products and customers feature group names

In this notebook, we will be using the feature groups created in Module 1 of the workshop. In case you would like to use a different feature group, please uncomment and replace **ORDERS_FEATURE_GROUP_NAME**, **CUSTOMERS_FEATURE_GROUP_NAME**, **PRODUCTS_FEATURE_GROUP_NAME** with the name of your Feature Groups for example. 

In [ ]:
# Retrieve FG names (when running previous modules)
%store -r customers_feature_group_name
%store -r orders_feature_group_name
%store -r products_feature_group_name

#orders_feature_group_name = '<ORDERS_FEATURE_GROUP_NAME>'
#customers_feature_group_name = '<CUSTOMERS_FEATURE_GROUP_NAME>'
#products_feature_group_name = '<PRODUCTS_FEATURE_GROUP_NAME>'

customers_fg = FeatureGroup(name=customers_feature_group_name, sagemaker_session=feature_store_session)  
orders_fg = FeatureGroup(name=orders_feature_group_name, sagemaker_session=feature_store_session)
products_fg = FeatureGroup(name=products_feature_group_name, sagemaker_session=feature_store_session)

#### Build Athena query to retrieve records from features groups - `customers` `products` `orders`

In [ ]:
customers_query = customers_fg.athena_query()
customers_table = customers_query.table_name

products_query = products_fg.athena_query()
products_table = products_query.table_name

orders_query = orders_fg.athena_query()
orders_table = orders_query.table_name

In [ ]:
orders_query_string = f'SELECT * FROM "sagemaker_featurestore"."{orders_table}" '
customers_query_string = f'SELECT * FROM "sagemaker_featurestore"."{customers_table}" '
products_query_string = f'SELECT * FROM "sagemaker_featurestore"."{products_table}" '

%store orders_query_string
orders_query_string

%store customers_query_string
customers_query_string

%store products_query_string
products_query_string

In [ ]:
output_location = f's3://{default_bucket}/athena-results/'
print(f'Athena query output location: \n{output_location}')

### Scenario 1: Row Level Access using Lake Formation using `orders`

#### Lakeformation applies row level access control and the sql query returns data only for one customer Id.

##### Extract the data from Feature Store using SageMaker SDK and Athena query

In [ ]:
orders_query.run(query_string=orders_query_string, output_location=output_location)
orders_query.wait()
orders_df = orders_query.as_dataframe()
orders_df.head()

### Scenario 2: Column Level Access using Lake Formation using `Products`
#### Lakeformation applies column level access control and excludes multiple columns from the resultset.

##### Extract the data from Feature Store using SageMaker SDK and Athena query

In [ ]:
products_query.run(query_string=products_query_string, output_location=output_location)
products_query.wait()
products_df = products_query.as_dataframe()
products_df.head()

### Scenario 3: Cell Level Access using Lake Formation using `Customers`
#### Lakeformation applies both row and column level ( also known as cell level) access control. The athena query result exclude two columns and return rows for a specific customer Id.

##### Extract the data from Feature Store using SageMaker SDK and Athena query

In [ ]:
customers_query.run(query_string=customers_query_string, output_location=output_location)
customers_query.wait()
customers_df = customers_query.as_dataframe()
customers_df.head()